In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the breast cancer dataset
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

# Extract features and target
X = data.data
y = data.target

# Function to normalize features
def normalize_features(X):
    X_normalized = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    return X_normalized

# Normalize features
X_normalized = normalize_features(X)

# Add intercept term
X_normalized = np.hstack((np.ones((X_normalized.shape[0], 1)), X_normalized))

# Divide dataset into training and testing sets (70:30 ratio)
np.random.seed(42)
indices = np.random.permutation(len(X_normalized))
split_index = int(0.7 * len(X_normalized))
X_train, X_test = X_normalized[indices[:split_index]], X_normalized[indices[split_index:]]
y_train, y_test = y[indices[:split_index]], y[indices[split_index:]]

# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Function to calculate logistic regression cost
def compute_cost(X, y, theta):
    m = len(y)
    h = sigmoid(X.dot(theta))
    cost = (-1 / m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
    return cost

# Function to perform logistic regression gradient descent
def gradient_descent(X, y, theta, learning_rate, iterations):
    m = len(y)
    cost_history = np.zeros(iterations)
    for i in range(iterations):
        h = sigmoid(X.dot(theta))
        gradient = (1 / m) * X.T.dot(h - y)
        theta -= learning_rate * gradient
        cost_history[i] = compute_cost(X, y, theta)
    return theta, cost_history

# Initialize theta and hyperparameters
theta = np.zeros(X_train.shape[1])
learning_rate = 0.01
iterations = 1000

# Perform gradient descent to train the model
theta_trained, cost_history = gradient_descent(X_train, y_train, theta, learning_rate, iterations)

# Function to make predictions
def predict(X, theta):
    probabilities = sigmoid(X.dot(theta))
    return [1 if p >= 0.5 else 0 for p in probabilities]

# Make predictions on training and testing sets
y_train_pred = predict(X_train, theta_trained)
y_test_pred = predict(X_test, theta_trained)

# Function to calculate confusion matrix
def confusion_matrix(actual, predicted):
    tp = np.sum((actual == 1) & (predicted == 1))
    tn = np.sum((actual == 0) & (predicted == 0))
    fp = np.sum((actual == 0) & (predicted == 1))
    fn = np.sum((actual == 1) & (predicted == 0))
    return tp, tn, fp, fn

# Calculate confusion matrix for training and testing sets
tp_train, tn_train, fp_train, fn_train = confusion_matrix(y_train, y_train_pred)
tp_test, tn_test, fp_test, fn_test = confusion_matrix(y_test, y_test_pred)

# Function to calculate sensitivity
def sensitivity(tp, fn):
    return tp / (tp + fn)

# Function to calculate specificity
def specificity(tn, fp):
    return tn / (tn + fp)

# Calculate sensitivity and specificity for training and testing sets
sensitivity_train = sensitivity(tp_train, fn_train)
specificity_train = specificity(tn_train, fp_train)
sensitivity_test = sensitivity(tp_test, fn_test)
specificity_test = specificity(tn_test, fp_test)

# Print confusion matrix, sensitivity, and specificity
print("Confusion Matrix for Training Set:")
print("TP:", tp_train, "TN:", tn_train, "FP:", fp_train, "FN:", fn_train)
print("Sensitivity (Training):", sensitivity_train)
print("Specificity (Training):", specificity_train)
print("\nConfusion Matrix for Testing Set:")
print("TP:", tp_test, "TN:", tn_test, "FP:", fp_test, "FN:", fn_test)
print("Sensitivity (Testing):", sensitivity_test)
print("Specificity (Testing):", specificity_test)

# Function to calculate accuracy
def accuracy(actual, predicted):
    return np.mean(actual == predicted)

# Calculate training and testing accuracies
training_accuracy = accuracy(y_train, y_train_pred)
testing_accuracy = accuracy(y_test, y_test_pred)

# Plotting training and testing accuracy vs iteration number
plt.plot(range(iterations), cost_history, color='blue')
plt.title('Training Accuracy vs Iteration Number')
plt.xlabel('Iteration Number')
plt.ylabel('Training Accuracy')
plt.show()

print("Training Accuracy:", training_accuracy)
print("Testing Accuracy:", testing_accuracy)
